# 1. [60+10] Language and Topic models

A common suggestion to users for coming up with good queries is to think of words that would likely appear in a relevant document, and to use those words as the query. The language modeling approach to IR directly models this idea: a document is a good match to a query if the document model is likely to generate the query, which will in turn happen if the document contains the query words often. 

You will score documents with respect to user query using language models and also get some experience with topic modelling.

## 1.0. [5] Loading data

We use the dataset we already used once - [this topic-modeling dataset](https://code.google.com/archive/p/topic-modeling-tool/downloads) ([or from github](https://github.com/IUCVLab/information-retrieval/blob/main/datasets/topic-modelling.zip)).

In [12]:
# TODO: read the dataset
import os
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
all_data = []

mapping = {"testdata_braininjury_10000docs.txt":"brain injury","testdata_news_music_2084docs.txt":"music","testdata_news_fuel_845docs.txt":"fuel","testdata_news_economy_2073docs.txt":"news_economy"}

for item in os.listdir('topic-modelling'):
    fn = os.path.join('topic-modelling',item)
    if os.path.isfile(fn):
        with open(fn, 'r') as fin:
            for doc in fin:
                doc = doc.lower()
                text_tokens = doc.translate(str.maketrans('', '', string.punctuation))
                all_data.append((text_tokens,mapping[item]))


print("# of documents", len(all_data))
assert len(all_data) == 15002

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maximevgrafov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# of documents 15002


## 1.1. [55] Ranking Using Language Models
Our goal is to rank documents by $P(d|q)$, where the probability of a document is interpreted as the likelihood that it is relevant to the query. 

Using Bayes rule: $P(d|q) = \frac{P(q|d)P(d)}{P(q)}$

$P(q)$ is the same for all documents, and so can be ignored. The prior probability of a document $P(d)$ is often treated as uniform across all $d$'s and so it can also be ignored. What does it mean? 

It means that computing $P(q|d)$ for different documents we can compare how relevant are they to the query. How can we estimate $P(q|d)$?

$P(q|d)$ can be estimated as:

![](https://i.imgur.com/BEIMAC1.png)

where $M_d$ is the language model of document $d$, $tf_{t,d}$ is the term frequency of term $t$ in document $d$, and $L_d$ is the number of tokens in document $d$. That is, we just count up how often each word occurred, and divide by the total number of words in the document $d$.

### 1.1.1. [15] Build TDM (or DTM)

The first thing we need to do is to build a term-document matrix for tour dataset.

In [13]:
# TODO: build term-document matrix for the dataset
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

documents = [doc for doc, topic in all_data]
vec = CountVectorizer()
X = vec.fit_transform(documents).astype(dtype=np.uint32)
terms = vec.get_feature_names()
tdm = X.toarray()

### 1.1.2. [30] Smoothing

Now, you need to implement the abovementioned logic in the `lm_rank_documents` function below. Do you see any potential problems?

Yes, data sparsity - we don't expect to meet each term in each doc, so, in most cases, we will get zero scores, which is not what we really want.

The solution is smooting.

One option is *[additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)* - adding a small number (0 to 1) to the observed counts and renormalizing to give a probability distribution.

Another option is called [Jelinek-Mercer smoothing](http://mlwiki.org/index.php/Smoothing_for_Language_Models#Jelinek-Mercer_Smoothing) - a simple idea that works well in practice is to use a mixture between a document-specific distribution and distribution estimated from the entire collection:

![](https://i.imgur.com/8Qv41Wp.png)

where 0 < λ < 1 and $M_c$ is a language model built from the entire document collection.

Refer to [*Chapter 12*](https://nlp.stanford.edu/IR-book/html/htmledition/language-models-for-information-retrieval-1.html) for the detailed explanation.


You are going to apply both in your `lm_rank_documents` function. This function takes TDM or DTM as an input, and ranks all documents "building" a language model for each document, returning relative probabilities of query being generated by a document as a document's score.

In [14]:
import numpy as np

def lm_rank_documents(query, tdm, smoothing='additive', param=0.001):
    # TODO: score each document in tdm using this document's language model
    # implement two types of smoothing. Looks up term frequencies in tdm
    # return document scores in a convenient form
    # param is alpha for additive / lambda for jelinek-mercer
    scores = [1]*len(documents)
    if smoothing == 'additive':
        d = len(terms)
        for doc_id in range(len(documents)):
            term_freqs = tdm[doc_id]
            doc_length = sum(term_freqs)
            for term in query:    
                try:
                    term_id = terms.index(term)
                except:
                    continue
                scores[doc_id]*=(term_freqs[term_id] + param)
            scores[doc_id] = scores[doc_id]/(doc_length + param*d)
   
    if smoothing == 'jelinek-mercer':
        d = sum(tdm[:,:])
        for doc_id in range(len(documents)):
            term_freqs = tdm[doc_id]
            doc_length = sum(term_freqs)
            md = 1
            mc = 1
            for term in query:    
                try:
                    term_id = terms.index(term)
                except:
                    continue
                md*=term_freqs[term_id]/doc_length
                mc*=sum(tdm[:,term_id].flatten())/d
            scores[doc_id] = param*md+(1-param)*mc
                    
    
    return scores

### 1.1.3. [10] Testing

Check if this type of ranking gives meaningful results. For each query output document `category`, `doc_id`, `score`, and the *beginning* of the document, as it is shown below. Analyze if categories and contents match the queries. 

In [19]:
def process_query(raw_query):
    # TODO: process user query and print search results including document category, id, score, and some part of it
    query = raw_query.lower().translate(str.maketrans('', '', string.punctuation)).split(' ')
    results = lm_rank_documents(query, tdm, smoothing='additive')
#     print(sorted(results,reverse=True)[:5])
    output_doc_ids = []
    for result in sorted(results,reverse=True)[:5]:
        doc_ids = [i for i, n in enumerate(results) if n == result]
        output_doc_ids += doc_ids
    
    output_doc_ids = sorted(list(set(output_doc_ids)), key=lambda x: results[x], reverse=True)
    
    for doc_id in output_doc_ids:
        print('\nScore:', results[doc_id], '\ndoc_id:',doc_id, '\ntopic:', all_data[doc_id][1], '\ndocument extract:',all_data[doc_id][0][:100])
    
#     for result in sorted(results,reverse=True)[:5]:
#         doc_id = results.index(result)
#         print('\nScore:',result, '\ndoc_id:',doc_id, '\ntopic:', all_data[doc_id][1], '\ndocument extract:',all_data[doc_id][0][:100])
    

user_queries = ["piano concert", "symptoms of head trauma", "wall street journal"]
for q in user_queries:
    print('User query:',q)
    process_query(q)
    print("\n")

User query: piano concert

Score: 0.03518347142755395 
doc_id: 10890 
topic: music 
document extract: playing carnegie hall concert performer dream when word leaked out that america most famous concert 

Score: 0.0323410692084542 
doc_id: 10916 
topic: music 
document extract: sometimes the most satisfying renovation the one that doesn happen two years ago geoffrey menin boug

Score: 0.031225391454488542 
doc_id: 10670 
topic: music 
document extract: igor kipnis the virtuoso harpsichordist whose busy concert recording career made him the instrument 

Score: 0.012865315730353695 
doc_id: 10810 
topic: music 
document extract: began said stephen somary not precisely with midsummer night dream with dream nonetheless would cond

Score: 0.009054029459686013 
doc_id: 11782 
topic: music 
document extract: could easily have been business usual the concert hall young ensemble not long out conservatory but 


User query: symptoms of head trauma

Score: 1.3254309856312645 
doc_id: 1998 
topic: 

## 1.2. [+10] Topic modeling

Now let's use *Latent Dirichlet Allocation* to identify topics in this collection and check if they match the original topics (fuel, economy, etc.). Go through the tutorial [here](https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0) and apply the ideas there to our dataset. 

In [20]:
# TODO: apply LDA to our dataset and output the resulting categories 
import gensim
from gensim.utils import simple_preprocess
import gensim.corpora as corpora
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

texts = {}
for doc, topic in all_data:
    if topic not in texts.keys():
        texts[topic] = []
    texts[topic].append(doc)
#"testdata_news_music_2084docs.txt":"music","testdata_news_fuel_845docs.txt":"fuel","testdata_news_economy_2073docs.txt":"news_economy"}
num_topics = 3
for key in list(texts):
    text = remove_stopwords(list(sent_to_words(texts[key])))
    id2word = corpora.Dictionary(text)
    corpus = [id2word.doc2bow(t) for t in text]
              
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
    print("Topic:", key)
    result = re.split('\W',str(lda_model.print_topics()))
    to_print = set()
    for r in result:
        if r.isalpha():
            to_print.add(r)
    print(to_print)    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/maximevgrafov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Topic: brain injury
{'brain', 'results', 'outcome', 'traumatic', 'injuries', 'injury', 'trauma', 'treatment', 'head', 'tbi', 'patient', 'study', 'patients'}
Topic: music
{'time', 'would', 'new', 'one', 'atlanta', 'first', 'york', 'also', 'people', 'said', 'like', 'two', 'year', 'music', 'news'}
Topic: fuel
{'bush', 'new', 'one', 'first', 'news', 'york', 'could', 'people', 'said', 'like', 'two', 'last', 'enron', 'year', 'would', 'company'}
Topic: news_economy
{'bush', 'new', 'would', 'atlanta', 'economy', 'one', 'york', 'percent', 'people', 'said', 'last', 'enron', 'year', 'news'}


We expect to see something like this (if collapsed, click on 3 dots):

```
Topic #0:
brain injury patients tbi traumatic study cerebral results severe group cognitive clinical pressure imaging following outcome control using children test

Topic #1:
new said york news atlanta like times year service time people undated just music journal constitution city says com years

Topic #2:
patients injury injuries trauma head study results traumatic brain treatment cases patient fractures years case outcome methods clinical tbi surgery

Topic #3:
said year bush percent new enron company president government people economy years million state companies states economic united time billion
```

# 2. [40+10] Sugges_

One of the strategies to improve user experience is to provide user with hints, or, otherwise, to autocomplete his queries. Let's consider suggest.

Today we will practice generating suggestions using [Trie](https://en.wikipedia.org/wiki/Trie) datastructure (prefix tree), see the example below.

Plan of your homework:

1. Build Trie based on real search query data, provided by AOL company;
2. Generate suggestion based on trie;
3. Measure suggestion speed;
4. (+) Optionally add spellcheck to suggest.


![image](https://www.ritambhara.in/wp-content/uploads/2017/05/Screen-Shot-2017-05-01-at-4.01.38-PM.png)

## 2.0. Install Trie data structure support

You are free to use any library implementation of Trie, as well as the one we suggest (read the docs before asking any questions!): https://github.com/google/pygtrie

In [ ]:
# !pip install pygtrie

### 2.0.1. Check it works and understand the example

In [22]:
import pygtrie
t = pygtrie.CharTrie()

# trie can be considered as a form of organizing a set of map
t["this is 1"] = "A"
t["this is 2"] = "B"
t["that is 3"] = "C"

print(t)

# "this" string is present in a set
n = t.has_node('this') == pygtrie.Trie.HAS_VALUE
# "this" prefix is present in a set
s = t.has_node('this') == pygtrie.Trie.HAS_SUBTRIE

print(f"Node = {n}\nSubtree = {s}")

# iterate a subtree
for key, val in t.iteritems("this"):
    print(key, '~', val)


CharTrie(this is 1: A, this is 2: B, that is 3: C)
Node = False
Subtree = True
this is 1 ~ A
this is 2 ~ B


## 2.1. Build a trie upon a dataset

### 2.1.1. [5] Read dataset

Download the [dataset](https://github.com/IUCVLab/information-retrieval/tree/main/datasets/aol) (we provide only the first part of the original data for simplicity (~3.5 mln queries)).

Explore the data, see readme file. Load the dataset.

In [23]:
import pandas as pd
aol_data = None

#TODO: Read the dataset, e.g. as pandas dataframe
aol_data = pd.read_csv("user-ct-test-collection-01.txt", encoding='utf-8', sep="\t")
assert aol_data.shape[0] == 3558411, "Dataset size doesnt' match"

### 2.1.2. [10] Build Trie

We want suggest function to be **non-sensitive to stop words** because we don't want to upset the user if he confuses/omits prepositions, for example. Consider `public events in Innopolis` vs `public events at Innopolis` or `public events Innopolis` - they all mean the same.

Build Trie based on the dataset, **storing query statistics such as query frequency, urls and ranks in nodes**. Some queries may not have associated urls, others may have multiple ranked urls, consider this.

In [24]:
aol_trie = pygtrie.CharTrie()
import re

query_stopwords = stopwords.words('english')
query_stopwords.remove('and')
query_stopwords.remove('the')
query_stopwords.remove('when')
query_stopwords.remove('you')

#TODO: build trie based on dataset
def query_to_words(sentence):
    sent = re.split('\W+', str(sentence))
    return sent
def rem_query_stopwds(text):
    return [word for word in text if word not in query_stopwords]

for _, item in aol_data.iterrows():
    raw_query = item.Query
    if raw_query is not None and str(raw_query) != 'nan' and len(str(raw_query)) > 0:
        query = " ".join(rem_query_stopwds(query_to_words(raw_query)))
        if (aol_trie.has_node(query) == pygtrie.Trie.HAS_VALUE):
            for key, val in aol_trie.iteritems(query):
                if item.ClickURL is not None and str(item.ClickURL) != 'nan' and len(str(item.ClickURL)) > 0:
                    aol_trie[key] = (val[0]+1,val[1]+[(item.ClickURL, item.ItemRank)])
                else:
                    aol_trie[key] = (val[0]+1,val[1])
        else:
            if item.ClickURL is not None and str(item.ClickURL) != 'nan' and len(str(item.ClickURL)) > 0:
                aol_trie[query] = (1,[(item.ClickURL, item.ItemRank)])
            else:
                aol_trie[query] = (1,[])
                

In [25]:
# test trie
bag = []
for key, val in aol_trie.iteritems("sample q"):
    print(key, '~', val)
    
    #NB: here we assume you store urls in a list field. But you can do something different. 
    bag += [url for url, _ in val[1]]
    
    assert "sample question" in key, "All examples have `sample question` substring"
    assert key[:len("sample question")] == "sample question", "All examples have `sample question` starting string"

for url in ["http://www.surveyconnect.com", "http://www.custominsight.com", 
            "http://jobsearchtech.about.com", "http://www.troy.k12.ny.us",
            "http://www.flinders.edu.au", "http://uscis.gov"]:
    assert url in bag, "This url should be in a try"

sample question surveys ~ (5, [('http://www.surveyconnect.com', 7.0), ('http://www.custominsight.com', 4.0), ('http://www.askemployees.com', 10.0), ('http://www.lg-employers.gov.uk', 1.0)])
sample questions immigration interview ~ (1, [])
sample questions interview ~ (1, [('http://www.quintcareers.com', 1.0)])
sample questions family interview ~ (3, [('http://www.grandparents-day.com', 2.0), ('http://www.quintcareers.com', 5.0), ('http://jobsearchtech.about.com', 3.0)])
sample questions sociology race and ethnicity ~ (1, [])
sample questions biology ~ (2, [('http://www.utexas.edu', 3.0), ('http://www.troy.k12.ny.us', 6.0)])
sample questions us citizenship test ~ (1, [('http://uscis.gov', 1.0)])
sample questionarie teaching evaluation ~ (1, [])
sample questionnaire teaching evaluation ~ (5, [('http://www.surveyconsole.com', 1.0), ('http://www.usask.ca', 2.0), ('http://teaching.berkeley.edu', 6.0), ('http://www.flinders.edu.au', 9.0), ('http://oregonstate.edu', 10.0)])
sample questionnai

## 2.2. [10] Write a suggest function which is non-sensitive to stop words

Suggest options for user query based on Trie you just built.
Output results sorted by frequency, print query count for each suggestion. If there is an url available, print the url too. If multiple url-s are available, print the one with the highest rank (the less the better).

In [26]:
def complete_user_query(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    query = " ".join(rem_query_stopwds(query_to_words(query)))
    if not (trie.has_node(query) == pygtrie.Trie.HAS_SUBTRIE or trie.has_node(query) == pygtrie.Trie.HAS_VALUE):
        return []
    results = []
    for key, val in trie.iteritems(query):
        results.append((key, val))
    if len(results)==0:
        return []
    results = [(query, val) for query, val in (sorted(results,key=lambda x: x[1][0],reverse=True))[:min(len(results),top_k)]]
    suggestions = []
    sug_urls = []
    for q,v in results:
        urls = v[1]
        suggestions.append(q)
        if len(urls)!=0:
            urls = sorted(urls, key=lambda x: x[1])
            sug_urls.append(urls[0][0])
        
    return suggestions, sug_urls

        
inp = "trie"
print("Query:", inp)
print("Results:")
res, urls = complete_user_query(inp, aol_trie)
print(res)
print(urls)

#NB we assume you return suggested query string only
assert res[0] == "tried and true tattoo"
assert res[1] == "triest" or res[1] == "triethanalomine"


Query: trie
Results:
['tried and true tattoo', 'triest', 'triethanalomine', 'tried and failed', 'tried and truechildren consignment sale']
['http://www.triedntruetattoo.com', 'http://avalon.unomaha.edu']


## 2.3. [10] Measure suggest speed ##

Check how fast your search is working. Consider changing your code if it takes too long on average.

In [27]:
import datetime
inp_queries = ["inf", "the best ", "information retrieval", "sherlock hol", "carnegie mell", 
               "babies r", "new york", "googol", "inter", "USA sta", "Barbara "]

#TODO: measure avg execution time (in milliseconds) per query and print it out
execution_times = [] 
for query in inp_queries:
    result = []
    start_time = datetime.datetime.now()
    if (aol_trie.has_node(query) == pygtrie.Trie.HAS_SUBTRIE or aol_trie.has_node(query) == pygtrie.Trie.HAS_VALUE):
        for key, val in aol_trie.iteritems(query):
            result.append((key, val))
    end_time = datetime.datetime.now()
    execution_times.append((end_time-start_time).total_seconds())

print(sum(execution_times)/len(execution_times))

0.00025872727272727267


## 2.4. [5] Question
What is the empirical threshold for minimal prefix length for suggest query? Write your answer in a cell bellow and justify it (just few sentences).

Since the majority of "serving" words such as prepositions, pronouns, articles are of length 2,3, and the word that convey meaning to the query are mostly of a length greater than 3, it is better to set the minimal threshold as 4. 

## 2.5. [+10] Add spellchecking to your suggest

Try to make your search results as close as possible. Compare top-5 results of each query with top-5 results for corrected.

In [29]:
inp_queries = ["inormation retrieval", "shelrock hol", "carnagie mell", "babis r", "Barrbara "]
inp_queries_corrected = ["information retrieval", "sherlock hol", "carnegie mell", "babies r", "Barbara "]

#!pip install autocorrect
from autocorrect import Speller

spell = Speller(lang='en')



def complete_user_query_with_spellchecker(query, trie, top_k=5):
    #TODO: suggest top_k options for a user query
    # sort results by frequency, suggest first ranked urls if available
    corrected_query = []
    for word in query.split(' '):
        if len(word)>4:
            corrected_query.append(spell(word))
        else:
            corrected_query.append(word)
    corrected_query = (" ".join(corrected_query))
    return complete_user_query(corrected_query, trie, top_k=top_k)


for q, qc in zip(inp_queries, inp_queries_corrected):
    assert  complete_user_query(qc, aol_trie, 5) == \
            complete_user_query_with_spellchecker(q, aol_trie, 5), "Assert {} and {} give different results".format(q, qc)